In [13]:
import numpy as np
import matplotlib.pyplot as plt
import torch

from Utils import test_env, utils
from Utils import HP_tuning 
from Utils.supervised import *

from RelationalModule import AC_networks as nets
from RelationalModule.MLP_AC_networks import Actor

from importlib import reload

parameters to try out and their priors, class for evaluation, model

In [14]:
reload(HP_tuning)

<module 'Utils.HP_tuning' from '/m/home/home9/94/dainesn1/unix/Workdir/RelationalDeepRL/Utils/HP_tuning.py'>

In [15]:
class EvalSandboxModel():
    def __init__(self, model, model_spec, game_params, n_epochs=50, n_samples=10000):
        self.model = model
        self.model_spec = model_spec
        self.game_params = game_params
        self.n_epochs = n_epochs
        self.n_samples = n_samples
        
    def evaluate_params(self, HPs, lambdas):
        train_V_lambda = [] 
        val_V_lambda = []
        for lr in lambdas:
            net = self.model(**self.model_spec, **HPs)
            results = supervised_training(net, lr, self.n_epochs, self.n_samples, 
                                          self.game_params, get_probs=True)
            trained_net, train_loss, val_loss, dataloader_dict, state_set, action_set, env = results
            
            train_V = 1-np.array(train_loss)
            train_V_lambda.append(train_V)
            
            val_V = 1-np.array(val_loss)
            val_V_lambda.append(val_V)
            
        return np.array(train_V_lambda), np.array(val_V_lambda)

In [16]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [2,2]
MAX_STEPS = 25

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

In [17]:
model = nets.MultiplicativeActor
model_spec = dict(action_space=4, linear_size = X+2, in_channels=3)

In [18]:
eval_instance = EvalSandboxModel(model, model_spec, game_params, 50, 10000)

In [19]:
value_priors_dict = {'out_channels':([[6],[8], [12],[6,12]], [1/4, 1/4, 1/4, 1/4]),
                     'max_pool_size':([2,3,4], [0.5, 0.3, 0.2]),
                     'n_features':([16,32,64],[1/3,1/3,1/3]),
                     'info_channels':([4,6,8],[0.2, 0.3, 0.5]),
                     'mask_channels':([4,6,8],[0.2, 0.3, 0.5]),
                     'hidden_channels':([6, 12, 32], [1/3, 1/3, 1/3]),
                     'residual_hidden_dim':([16,32,64],[1/3, 1/3, 1/3]),
                     'n_residual_layers':([1,2,3,4],[0.1, 0.3, 0.3, 0.3])}

In [20]:
bayes_HP_tuning = HP_tuning.BayesHPTuning(value_priors_dict, eval_instance)

In [21]:
for i in range(100):
    bayes_HP_tuning.step()


Configuration sampled: 
	out_channels :  [12]  - prob: 0.25
	max_pool_size :  4  - prob: 0.20
	n_features :  64  - prob: 0.33
	info_channels :  8  - prob: 0.50
	mask_channels :  8  - prob: 0.50
	hidden_channels :  12  - prob: 0.33
	residual_hidden_dim :  16  - prob: 0.33
	n_residual_layers :  4  - prob: 0.30

Creating dataset...

Training network...
Verbose:  True
Using cuda:  False
Epoch 1, 100% 	 Train loss: 0.2639 took: 2.02s  Val. loss: 0.2569
Epoch 2, 100% 	 Train loss: 0.2528 took: 2.00s  Val. loss: 0.2258
Epoch 3, 100% 	 Train loss: 0.2097 took: 2.01s  Val. loss: 0.1995
Epoch 4, 100% 	 Train loss: 0.1956 took: 2.00s  Val. loss: 0.1896
Epoch 5, 100% 	 Train loss: 0.1901 took: 1.99s  Val. loss: 0.1828
Epoch 6, 100% 	 Train loss: 0.1889 took: 1.99s  Val. loss: 0.1828
Epoch 7, 100% 	 Train loss: 0.1871 took: 1.98s  Val. loss: 0.1817
Epoch 8, 100% 	 Train loss: 0.1857 took: 1.99s  Val. loss: 0.1790
Epoch 9, 100% 	 Train loss: 0.1848 took: 1.97s  Val. loss: 0.1797
Epoch 10, 100% 	 Tr

Epoch 13, 100% 	 Train loss: 0.2037 took: 2.01s  Val. loss: 0.2184
Epoch 14, 100% 	 Train loss: 0.2011 took: 2.00s  Val. loss: 0.2174
Epoch 15, 100% 	 Train loss: 0.1998 took: 2.00s  Val. loss: 0.2126
Epoch 16, 100% 	 Train loss: 0.1992 took: 1.99s  Val. loss: 0.2142
Epoch 17, 100% 	 Train loss: 0.1990 took: 2.01s  Val. loss: 0.2115
Epoch 18, 100% 	 Train loss: 0.1969 took: 1.99s  Val. loss: 0.2102
Epoch 19, 100% 	 Train loss: 0.1984 took: 2.00s  Val. loss: 0.2124
Epoch 20, 100% 	 Train loss: 0.1951 took: 2.02s  Val. loss: 0.2153
Epoch 21, 100% 	 Train loss: 0.1947 took: 2.00s  Val. loss: 0.2137
Epoch 22, 100% 	 Train loss: 0.1952 took: 1.18s  Val. loss: 0.2252
Epoch 23, 100% 	 Train loss: 0.1972 took: 1.18s  Val. loss: 0.2099
Epoch 24, 100% 	 Train loss: 0.1949 took: 1.19s  Val. loss: 0.2187
Epoch 25, 100% 	 Train loss: 0.1935 took: 1.18s  Val. loss: 0.2098
Epoch 26, 100% 	 Train loss: 0.1946 took: 1.18s  Val. loss: 0.2124
Epoch 27, 100% 	 Train loss: 0.1926 took: 1.18s  Val. loss: 0.

Epoch 18, 100% 	 Train loss: 0.1551 took: 1.06s  Val. loss: 0.1626
Epoch 19, 100% 	 Train loss: 0.1538 took: 1.07s  Val. loss: 0.1608
Epoch 20, 100% 	 Train loss: 0.1545 took: 1.07s  Val. loss: 0.1634
Epoch 21, 100% 	 Train loss: 0.1537 took: 1.06s  Val. loss: 0.1607
Epoch 22, 100% 	 Train loss: 0.1521 took: 1.07s  Val. loss: 0.1642
Epoch 23, 100% 	 Train loss: 0.1522 took: 1.07s  Val. loss: 0.1586
Epoch 24, 100% 	 Train loss: 0.1516 took: 1.06s  Val. loss: 0.1638
Epoch 25, 100% 	 Train loss: 0.1505 took: 1.06s  Val. loss: 0.1613
Epoch 26, 100% 	 Train loss: 0.1488 took: 1.07s  Val. loss: 0.1593
Epoch 27, 100% 	 Train loss: 0.1496 took: 1.06s  Val. loss: 0.1588
Epoch 28, 100% 	 Train loss: 0.1475 took: 1.07s  Val. loss: 0.1591
Epoch 29, 100% 	 Train loss: 0.1466 took: 1.08s  Val. loss: 0.1523
Epoch 30, 100% 	 Train loss: 0.1469 took: 1.10s  Val. loss: 0.1565
Epoch 31, 100% 	 Train loss: 0.1458 took: 1.11s  Val. loss: 0.1579
Epoch 32, 100% 	 Train loss: 0.1449 took: 1.13s  Val. loss: 0.

Epoch 23, 100% 	 Train loss: 0.0919 took: 1.01s  Val. loss: 0.1066
Epoch 24, 100% 	 Train loss: 0.0907 took: 1.02s  Val. loss: 0.1060
Epoch 25, 100% 	 Train loss: 0.0893 took: 1.02s  Val. loss: 0.1098
Epoch 26, 100% 	 Train loss: 0.0884 took: 1.02s  Val. loss: 0.1042
Epoch 27, 100% 	 Train loss: 0.0848 took: 1.02s  Val. loss: 0.1014
Epoch 28, 100% 	 Train loss: 0.0842 took: 1.02s  Val. loss: 0.1021
Epoch 29, 100% 	 Train loss: 0.0828 took: 1.02s  Val. loss: 0.1012
Epoch 30, 100% 	 Train loss: 0.0827 took: 1.03s  Val. loss: 0.0966
Epoch 31, 100% 	 Train loss: 0.0812 took: 1.03s  Val. loss: 0.1007
Epoch 32, 100% 	 Train loss: 0.0815 took: 1.04s  Val. loss: 0.1036
Epoch 33, 100% 	 Train loss: 0.0806 took: 1.08s  Val. loss: 0.1022
Epoch 34, 100% 	 Train loss: 0.0798 took: 1.11s  Val. loss: 0.0996
Epoch 35, 100% 	 Train loss: 0.0788 took: 1.11s  Val. loss: 0.0973
Epoch 36, 100% 	 Train loss: 0.0795 took: 1.10s  Val. loss: 0.0996
Epoch 37, 100% 	 Train loss: 0.0782 took: 1.10s  Val. loss: 0.

Epoch 42, 100% 	 Train loss: 0.1633 took: 1.05s  Val. loss: 0.1643
Epoch 43, 100% 	 Train loss: 0.1647 took: 1.06s  Val. loss: 0.1675
Epoch 44, 100% 	 Train loss: 0.1628 took: 1.06s  Val. loss: 0.1642
Epoch 45, 100% 	 Train loss: 0.1628 took: 1.06s  Val. loss: 0.1646
Epoch 46, 100% 	 Train loss: 0.1625 took: 1.05s  Val. loss: 0.1618
Epoch 47, 100% 	 Train loss: 0.1619 took: 1.06s  Val. loss: 0.1641
Epoch 48, 100% 	 Train loss: 0.1612 took: 1.05s  Val. loss: 0.1627
Epoch 49, 100% 	 Train loss: 0.1618 took: 1.05s  Val. loss: 0.1639
Epoch 50, 100% 	 Train loss: 0.1605 took: 1.05s  Val. loss: 0.1643
Training finished, took 81.18s

Configuration sampled: 
	out_channels :  [6]  - prob: 0.25
	max_pool_size :  3  - prob: 0.31
	n_features :  64  - prob: 0.33
	info_channels :  8  - prob: 0.46
	mask_channels :  4  - prob: 0.22
	hidden_channels :  12  - prob: 0.33
	residual_hidden_dim :  64  - prob: 0.33
	n_residual_layers :  2  - prob: 0.32
lambda: 0.0050 - V: 0.876124
lambda: 0.0010 - V: 0.83681

...

Epoch 19, 100% 	 Train loss: 0.1659 took: 1.03s  Val. loss: 0.1649
Epoch 20, 100% 	 Train loss: 0.1649 took: 1.03s  Val. loss: 0.1622
Epoch 21, 100% 	 Train loss: 0.1632 took: 1.03s  Val. loss: 0.1640
Epoch 22, 100% 	 Train loss: 0.1633 took: 1.03s  Val. loss: 0.1643
Epoch 23, 100% 	 Train loss: 0.1624 took: 1.03s  Val. loss: 0.1688
Epoch 24, 100% 	 Train loss: 0.1643 took: 1.04s  Val. loss: 0.1619
Epoch 25, 100% 	 Train loss: 0.1621 took: 1.02s  Val. loss: 0.1598
Epoch 26, 100% 	 Train loss: 0.1611 took: 1.03s  Val. loss: 0.1601
Epoch 27, 100% 	 Train loss: 0.1610 took: 1.03s  Val. loss: 0.1587
Epoch 28, 100% 	 Train loss: 0.1595 took: 1.12s  Val. loss: 0.1656
Epoch 29, 100% 	 Train loss: 0.1592 took: 1.04s  Val. loss: 0.1599
Epoch 30, 100% 	 Train loss: 0.1586 took: 1.06s  Val. loss: 0.1660
Epoch 31, 100% 	 Train loss: 0.1606 took: 1.06s  Val. loss: 0.1628
Epoch 32, 100% 	 Train loss: 0.1587 took: 1.06s  Val. loss: 0.1617
Epoch 33, 100% 	 Train loss: 0.1581 took: 1.06s  Val. loss: 0.

Epoch 24, 100% 	 Train loss: 0.2570 took: 1.43s  Val. loss: 0.2531
Epoch 25, 100% 	 Train loss: 0.2570 took: 1.43s  Val. loss: 0.2523
Epoch 26, 100% 	 Train loss: 0.2569 took: 1.44s  Val. loss: 0.2532
Epoch 27, 100% 	 Train loss: 0.2569 took: 1.44s  Val. loss: 0.2524
Epoch 28, 100% 	 Train loss: 0.2570 took: 1.45s  Val. loss: 0.2529
Epoch 29, 100% 	 Train loss: 0.2570 took: 1.45s  Val. loss: 0.2518
Epoch 30, 100% 	 Train loss: 0.2569 took: 1.47s  Val. loss: 0.2523
Epoch 31, 100% 	 Train loss: 0.2570 took: 1.46s  Val. loss: 0.2533
Epoch 32, 100% 	 Train loss: 0.2570 took: 1.47s  Val. loss: 0.2527
Epoch 33, 100% 	 Train loss: 0.2569 took: 1.48s  Val. loss: 0.2531
Epoch 34, 100% 	 Train loss: 0.2569 took: 2.41s  Val. loss: 0.2528
Epoch 35, 100% 	 Train loss: 0.2570 took: 2.45s  Val. loss: 0.2525
Epoch 36, 100% 	 Train loss: 0.2570 took: 2.42s  Val. loss: 0.2524
Epoch 37, 100% 	 Train loss: 0.2570 took: 2.42s  Val. loss: 0.2525
Epoch 38, 100% 	 Train loss: 0.2570 took: 2.43s  Val. loss: 0.

Epoch 29, 100% 	 Train loss: 0.1588 took: 1.93s  Val. loss: 0.1636
Epoch 30, 100% 	 Train loss: 0.1561 took: 1.95s  Val. loss: 0.1602
Epoch 31, 100% 	 Train loss: 0.1568 took: 1.96s  Val. loss: 0.1595
Epoch 32, 100% 	 Train loss: 0.1560 took: 2.04s  Val. loss: 0.1585
Epoch 33, 100% 	 Train loss: 0.1540 took: 2.18s  Val. loss: 0.1610
Epoch 34, 100% 	 Train loss: 0.1533 took: 2.19s  Val. loss: 0.1545
Epoch 35, 100% 	 Train loss: 0.1524 took: 2.19s  Val. loss: 0.1590
Epoch 36, 100% 	 Train loss: 0.1538 took: 2.20s  Val. loss: 0.1571
Epoch 37, 100% 	 Train loss: 0.1516 took: 2.23s  Val. loss: 0.1544
Epoch 38, 100% 	 Train loss: 0.1528 took: 2.25s  Val. loss: 0.1542
Epoch 39, 100% 	 Train loss: 0.1509 took: 2.25s  Val. loss: 0.1512
Epoch 40, 100% 	 Train loss: 0.1502 took: 2.26s  Val. loss: 0.1539
Epoch 41, 100% 	 Train loss: 0.1501 took: 2.27s  Val. loss: 0.1563
Epoch 42, 100% 	 Train loss: 0.1496 took: 2.27s  Val. loss: 0.1548
Epoch 43, 100% 	 Train loss: 0.1500 took: 2.27s  Val. loss: 0.

Epoch 48, 100% 	 Train loss: 0.1796 took: 1.95s  Val. loss: 0.1901
Epoch 49, 100% 	 Train loss: 0.1793 took: 1.15s  Val. loss: 0.1901
Epoch 50, 100% 	 Train loss: 0.1792 took: 1.16s  Val. loss: 0.1909
Training finished, took 98.86s

Configuration sampled: 
	out_channels :  [6]  - prob: 0.25
	max_pool_size :  4  - prob: 0.32
	n_features :  32  - prob: 0.34
	info_channels :  8  - prob: 0.35
	mask_channels :  8  - prob: 0.34
	hidden_channels :  12  - prob: 0.33
	residual_hidden_dim :  16  - prob: 0.33
	n_residual_layers :  3  - prob: 0.26
lambda: 0.0050 - V: 0.824358
lambda: 0.0010 - V: 0.786590
lambda: 0.0005 - V: 0.799329
Average V: 0.803426
Time elapsed: 308.60 s

Configuration sampled: 
	out_channels :  [12]  - prob: 0.26
	max_pool_size :  3  - prob: 0.33
	n_features :  32  - prob: 0.34
	info_channels :  8  - prob: 0.35
	mask_channels :  4  - prob: 0.32
	hidden_channels :  6  - prob: 0.34
	residual_hidden_dim :  32  - prob: 0.33
	n_residual_layers :  1  - prob: 0.23

Creating dataset.

Epoch 2, 100% 	 Train loss: 0.2545 took: 1.70s  Val. loss: 0.2571
Epoch 3, 100% 	 Train loss: 0.2544 took: 1.70s  Val. loss: 0.2571
Epoch 4, 100% 	 Train loss: 0.2543 took: 1.70s  Val. loss: 0.2569
Epoch 5, 100% 	 Train loss: 0.2543 took: 1.70s  Val. loss: 0.2558
Epoch 6, 100% 	 Train loss: 0.2537 took: 1.70s  Val. loss: 0.2562
Epoch 7, 100% 	 Train loss: 0.2524 took: 1.70s  Val. loss: 0.2535
Epoch 8, 100% 	 Train loss: 0.2468 took: 1.73s  Val. loss: 0.2414
Epoch 9, 100% 	 Train loss: 0.2204 took: 1.70s  Val. loss: 0.2036
Epoch 10, 100% 	 Train loss: 0.1928 took: 1.72s  Val. loss: 0.1838
Epoch 11, 100% 	 Train loss: 0.1761 took: 1.71s  Val. loss: 0.1710
Epoch 12, 100% 	 Train loss: 0.1668 took: 1.73s  Val. loss: 0.1653
Epoch 13, 100% 	 Train loss: 0.1658 took: 1.70s  Val. loss: 0.1637
Epoch 14, 100% 	 Train loss: 0.1624 took: 1.71s  Val. loss: 0.1609
Epoch 15, 100% 	 Train loss: 0.1602 took: 1.72s  Val. loss: 0.1628
Epoch 16, 100% 	 Train loss: 0.1598 took: 1.72s  Val. loss: 0.1623
Epo

In [22]:
HPs = bayes_HP_tuning.return_best_model()

In [23]:
HPs

{'out_channels': [12],
 'max_pool_size': 2,
 'n_features': 32,
 'info_channels': 8,
 'mask_channels': 8,
 'hidden_channels': 6,
 'residual_hidden_dim': 64,
 'n_residual_layers': 3}

In [24]:
for k in HPs:
    values, probs = bayes_HP_tuning.return_param_probs(k)
    print("\nParam: "+k)
    print("Values: ", values)
    print("Probs: ", probs)


Param: out_channels
Values:  [[6], [8], [12], [6, 12]]
Probs:  [0.25266946 0.25565142 0.25581348 0.23586564]

Param: max_pool_size
Values:  [2, 3, 4]
Probs:  [0.34545893 0.33361748 0.32092358]

Param: n_features
Values:  [16, 32, 64]
Probs:  [0.32978766 0.33626912 0.33394322]

Param: info_channels
Values:  [4, 6, 8]
Probs:  [0.31929414 0.33470014 0.34600572]

Param: mask_channels
Values:  [4, 6, 8]
Probs:  [0.32431845 0.33287537 0.34280617]

Param: hidden_channels
Values:  [6, 12, 32]
Probs:  [0.33557636 0.33308941 0.33133423]

Param: residual_hidden_dim
Values:  [16, 32, 64]
Probs:  [0.33089575 0.33217632 0.33692793]

Param: n_residual_layers
Values:  [1, 2, 3, 4]
Probs:  [0.23319344 0.25338094 0.25857873 0.2548469 ]


In [25]:
import string
import random

save = True
keywords = ['bayesian'] # example

if save:
    save_dir = 'Results/Sandbox/Supervised/'
    ID = ''.join([random.choice(string.ascii_letters) for _ in range(4)])
    ID = ID.upper()
    keywords.append(ID)
    filename = '_'.join(keywords)
    filename = 'S_'+filename
    print("Save at "+save_dir+"HP_tuning__"+filename)
    torch.save(bayes_HP_tuning, save_dir+"HP_tuning_"+filename)
else:
    print("Nothing saved")
    pass

Save at Results/Sandbox/Supervised/HP_tuning__S_bayesian_CAWN


/m/work/modules/automatic/anaconda/envs/aalto-ubuntu1804-generic/software/anaconda/2020-01-tf2/5a34a04a/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type MultiplicativeActor. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
